In [ ]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing


In [ ]:
data = pd.read_excel('/content/drive/My Drive/4-hate_speech_mlma_ar_dataset.xlsx')
print("Sample from dataset",data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3353 entries, 0 to 3352
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   tweet                3345 non-null   object
 1   sentiment            3353 non-null   object
 2   directness           3353 non-null   object
 3   annotator_sentiment  3353 non-null   object
 4   target               3353 non-null   object
 5   group                3353 non-null   object
dtypes: object(6)
memory usage: 157.3+ KB
Sample from dataset None


In [ ]:
data.head()

,tweet,sentiment,directness,annotator_sentiment,target,group
0,صلاة الفجر خير لك من ترديد بول البعير وسبي الن...,hateful_normal,indirect,shock,gender,individual
1,صراحة نفسي اشوف ولاد الوسخة اللي قالوا مدرب اج...,offensive,indirect,anger_confusion_sadness_indifference_disgust,other,other
2,طيب! هي متبرجة وعبايتها ملونه وطالعة من بيتهم ...,offensive,indirect,indifference,other,individual
3,@user @user انا اوافقك بخصوص السوريين و العراق...,normal,direct,indifference,origin,other
4,هذه السعودية التي شعبها شعب الخيم و بول البعير...,normal,indirect,indifference,origin,other


In [ ]:
data.isna().sum()

tweet                  8
sentiment              0
directness             0
annotator_sentiment    0
target                 0
group                  0
dtype: int64

In [ ]:
data.shape

(3353, 6)

In [ ]:
data.dropna(axis=0,inplace=True)

In [ ]:
data.shape

(3345, 6)

In [ ]:
data["tweet"].duplicated().sum()

12

In [ ]:
bool_series = data["tweet"].duplicated() 
data[bool_series][:12]

,tweet,sentiment,directness,annotator_sentiment,target,group
253,تحمل اللاجئون السوريون شدة البرد والحر,normal,direct,sadness,origin,refugees
803,فتوى جديدة من #العراق يجوز شرب بول البعير وبول...,abusive_disrespectful,indirect,indifference_disgust,origin,other
1309,على مهلك خدينك بحلمك لوين يا رامز لجوا له ولي ...,offensive,indirect,indifference,origin,other
1696,يبدو أنّك تحت تأثير بول البعير,abusive_disrespectful,direct,indifference,origin,individual
1823,في الزلفي بايرة المرأة التي لم تتزوج أو التي ل...,offensive,indirect,indifference,other,women
2096,معاك ريال تسوين ريال,disrespectful,indirect,sadness,other,individual
2122,وانتي متبرجة مثيرة للشفقة,hateful_disrespectful_normal,direct,indifference,other,individual
2175,ما بدي كون متطفّل,normal,direct,indifference,other,other
2263,لا للتصنيف عندما يتوقف تصنيف التي لم تتزوج بال...,normal,indirect,shock,gender,women
2519,مع إني ضد الشتيمة,normal,direct,indifference,origin,individual


In [ ]:
data.loc[data['tweet'] == "تحمل اللاجئون السوريون شدة البرد والحر"]

,tweet,sentiment,directness,annotator_sentiment,target,group
151,تحمل اللاجئون السوريون شدة البرد والحر,normal,direct,sadness_indifference,origin,refugees
253,تحمل اللاجئون السوريون شدة البرد والحر,normal,direct,sadness,origin,refugees


In [ ]:
# Drop rows that contain empty tweet
print("data_is_null \n",data.isnull().sum())
data=data[data.tweet.str.contains('#NAME?',na=False)]
data.info()

data_is_null 
 Unnamed: 0             0
tweet                  8
sentiment              0
directness             0
annotator_sentiment    0
target                 0
group                  0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           0 non-null      int64 
 1   tweet                0 non-null      object
 2   sentiment            0 non-null      object
 3   directness           0 non-null      object
 4   annotator_sentiment  0 non-null      object
 5   target               0 non-null      object
 6   group                0 non-null      object
dtypes: int64(1), object(6)
memory usage: 0.0+ bytes


In [ ]:
data = pd.read_excel('/content/drive/My Drive/Master thesis/DataSet-forUsing/4-hate_speech_mlma_ar_dataset.xlsx',usecols=['tweet', 'sentiment'])

data["HS_NOT"] = data.apply(lambda x: "Not-HS" if x["sentiment"] in ['normal'] else "HS",axis =1)

data.info()
data.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3353 entries, 0 to 3352
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet      3345 non-null   object
 1   sentiment  3353 non-null   object
dtypes: object(2)
memory usage: 52.5+ KB


(3353, 2)

In [ ]:
# data.drop(columns=['sentiment'])


In [ ]:
import seaborn as sns

sns.countplot(data.HS_NOT)
plt.xlabel('Label')
plt.title('Number of Hate and Normal tweets')

AttributeError: ignored

In [ ]:
# texts = pd.concat([data['tweet']],axis=0).astype("str")   
# from keras.preprocessing.text import Tokenize
from keras.preprocessing.text import Tokenizer
# print(texts)
print(data.shape)

tokeniser = Tokenizer(num_words=30000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', 
lower=True, split=' ', char_level=False, oov_token=None, document_count=0)




tokeniser.fit_on_texts(data['tweet'].astype("str"))
X = tokeniser.texts_to_sequences(data['tweet'].astype("str").values)

# pad: to make all input of same length
X = pad_sequences(X)

print(X[:5])
print(X.shape)
# Y.shape

(3353, 2)
[[   0    0    0    0    0    0    0    0    0    0    0    0    0 1958
  1324 1325   90    3 3933    4    5 1959  236 1960 1961 3934 3935 3936
  3937]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
   993  171  290  616 1326   24  617 1962 1327  141  754  324 3938  755
     2]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0  172   64   30 3939 3940 3941    3 3942  215
     2]
 [   0    0    0    0    0    0    0    0    0    0    0    1    1   34
  3943  756  757   10  618  142  756  325   65 1963   61 3944 1328 3945
     2]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0  105   97  101 3946   86 3947   10    4    5   96 3948  158
   102]]
(3353, 29)


In [ ]:
print((data['HS_NOT']).values)
Y = pd.get_dummies(data['HS_NOT']).values
# X = pd.get_dummies(data['tweet']).values
print(Y)
# X = tf.reshape(X, [1,-1])
# X_train = np.resize(X_train,(X_train.shape[0],1,X_train.shape[1]))

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.15, random_state = 42)
# trainX = mat([[1,1], [0,0]])
# trainY = array([1,0])
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
# matmul(X,Y)

['HS' 'HS' 'HS' ... 'HS' 'HS' 'Not-HS']
[[1 0]
 [1 0]
 [1 0]
 ...
 [1 0]
 [1 0]
 [0 1]]
(2850, 29) (2850, 2)
(503, 29) (503, 2)


In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=150)
    layer = Embedding(50,50,input_length=150)(inputs)
    # model.add(Bidirectional(GRU(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1)))
    layer = Bidirectional(GRU(128))(layer)
    layer = Dropout(0.5)(layer)
    # layer = LSTM(64)(layer)
    # layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(2,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model


In [ ]:
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.models import Model
from keras.optimizers import RMSprop
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional

model = RNN()
model.summary()
# opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6) 
# model.compile( loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])


model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 150)]             0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 150, 50)           2500      
_________________________________________________________________
bidirectional_6 (Bidirection (None, 256)               138240    
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
activation_10 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 2)               

In [ ]:
history=model.fit(X_train, Y_train, epochs=7, batch_size=29)


Epoch 1/7


InvalidArgumentError: ignored

In [ ]:
embed_dim = 128
lstm_out = 196

####
# The LSTM architecture
model=Sequential()
model.add(Embedding(2000,512,mask_zero=True,input_length= X.shape[1]))
# model.add(Bidirectional(LSTM(100, return_sequences=True, input_dim= 25 )))
model.add(Bidirectional(GRU(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1)))
# model.add(LSTM(256,dropout=0.1, recurrent_dropout=0.1,return_sequences=False))
# model.add(Dense(2,input_shape=(9, 3), activation='softmax'))
# model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# Compiling the RNN

# Fitting to the training set
# regressor.fit(X_train,Y_train,epochs=10,batch_size=32, verbose = 2)
model.fit(X_train, Y_train, epochs=4, batch_size=100, verbose=2)

# summary
print(model.summary())

Epoch 1/4


ValueError: ignored

In [ ]:
twt = ['انا اوافقك بخصوص السوريين و العراقيين اما بخصوص السعودي مو بحاجه انه يقعد بالكويت بدو']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokeniser.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("hate")
elif (np.argmax(sentiment) == 1):
    print("normal")

In [ ]:
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional
from keras.optimizers import Adam

model3= Sequential()
model3.add(Embedding(2000,100,input_length= X.shape[1]))
model3.add(Conv1D(64,kernel_size=3,padding='same',activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Dropout(0.25))
model3.add(GRU(128,return_sequences=True))
model3.add(Dropout(0.3))
model3.add(Flatten())
model3.add(Dense(128,activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(2,activation='softmax'))
model3.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model3.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 29, 100)           200000    
_________________________________________________________________
conv1d (Conv1D)              (None, 29, 64)            19264     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 14, 64)            0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 14, 64)            0         
_________________________________________________________________
gru_8 (GRU)                  (None, 14, 128)           74496     
_________________________________________________________________
dropout_13 (Dropout)         (None, 14, 128)           0         
_________________________________________________________________
flatten (Flatten)            (None, 1792)             